# Transformers AutoClass 

### Tokenizer, Model Loading
- Huggingface 모델 허브에서 제공하는 처리 모델을 다운받아 로딩한다.
- 다운로드된 모델은 `사용자 home 디렉토리\.cache\huggingface` 에 저장된다.
- 미리 학습된 언어 모델을 다운받아 사용할 때는 그 언어모델이 사용한 tokenizer를 같이 받아서 사용한다.

### [Auto Classes](https://huggingface.co/docs/transformers/model_doc/auto)
- Huggingface 에서 제공하는 다양한 모델들은 손쉽게 불러오고 사용할 수 있도록 설계된 유틸리티 클래스들을 말한다.
- 미리 학습된 특정 모델의 이름(모델 허브상에서 제공되는 이름)이나 저장된 local 경로를 제공하면 해당 모델에 맞는 적절한 클래스와 구성 요소를 자동으로 로드한다.
- 사용자는 모델을 사용하기 위한 정확한 클래스를 몰라도 쉽게 다양한 종류의 모델을 사용할 수있다.

#### 주요 Auto Class
- 기본 모델 Loading
    1. **AutoModel**
       - 주어진 모델 이름에 맞는 사전 학습된 모델을 자동으로 로드한다.
       - 예: `AutoModel.from_pretrained("bert-base-uncased")`: BERT 모델을 로드한다.
    2. **AutoTokenizer**
       - 해당 모델에 적합한 토크나이저를 자동으로 로드한다.
       - 예: `AutoTokenizer.from_pretrained("bert-base-uncased")`: BERT 모델에 맞는 토크나이저를 로드한다.
    3. **AutoConfig**
       - 모델의 설정(config)을 자동으로 로드한다. 모델 설정에는 모델의 하이퍼파라미터와 모델 구조 정보가 포함된다. 이 설정을 이용해 모델을 생성할 수있다.
       - 예: `AutoConfig.from_pretrained("bert-base-uncased")`
- Task 처리 모델 Loading
    - Pretrained backbone 모델에 각 task 에 맞는 estimator layer를 추가한 모델을 생성해 제공한다.
    - 주요 모델들
        1. **AutoModelForSequenceClassification**
           - 시퀀스(Text) 분류 작업을 위한 모델을 자동으로 로드한다.
           - 예: `AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")`
        2. **AutoModelForQuestionAnswering**
           - 질문-응답 작업을 위한 모델을 자동으로 로드한다.
           - 예: `AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")`
        3. **AutoModelForTokenClassification**
           - 토큰 분류 작업(예: 개체명 인식)을 위한 모델을 자동으로 로드한다.
           - 예: `AutoModelForTokenClassification.from_pretrained("bert-base-uncased")`

In [1]:
!uv pip install streamlit

Resolved 38 packages in 1.22s
Prepared 5 packages in 937ms
Installed 18 packages in 874ms
 + altair==6.0.0
 + blinker==1.9.0
 + cachetools==6.2.2
 + click==8.3.1
 + gitdb==4.0.12
 + gitpython==3.1.45
 + jsonschema==4.25.1
 + jsonschema-specifications==2025.9.1
 + narwhals==2.13.0
 + protobuf==6.33.2
 + pydeck==0.9.1
 + referencing==0.37.0
 + rpds-py==0.30.0
 + smmap==5.0.2
 + streamlit==1.52.1
 + tenacity==9.1.2
 + toml==0.10.2
 + watchdog==6.0.0


In [5]:
from transformers import AutoModel, AutoConfig, AutoTokenizer

model_id = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_id)
print(type(tokenizer))

model = AutoModel.from_pretrained(model_id)
print(type(model))

config = AutoConfig.from_pretrained(model_id)
print(type(config))

<class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>
<class 'transformers.models.bert.modeling_bert.BertModel'>
<class 'transformers.models.bert.configuration_bert.BertConfig'>


In [ ]:
#config #config.json 모델 아키텍쳐 관련 정보파일

model2 = AutoModel.from_config(config) # 위와는 다르게 학습이 되지 않은 모델
type(model2)

transformers.models.bert.modeling_bert.BertModel

In [ ]:
from transformers import BertModel, GPT2Model
b_model = BertModel.from_pretrained(model_id)
b_model

In [ ]:
###################
# raw - text ==> (Tokenizer) ==> token ids ==> (model) ==> 결과

In [18]:
# 토크나이저
raw_text = "I am a boy."
# 토큰화
token = tokenizer(raw_text)

token

{'input_ids': [101, 1045, 2572, 1037, 2879, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [ ]:
# token(결과)의 key 들 조회
dict(token).keys()
# 모델에 들어가야 하는 값들.


# 'input_ids' : 입력 토큰 id 값
# 'token_type_ids' : 입력이 문장 쌍일 때 문장을 구분하는 id. 0: 첫번째 문장, 1: 두번 째 문장
#                   각 토큰이 어느 문장에 속했는지 토큰 별로 지정됨
#                   문장 쌍 : QA(질문-지문)
# 'attention_mask' : 실제 문장의 토큰과 Padding을 구분한다. 0: padding, 1: 실제 문장 토큰

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
# 개별 조회
type(token['input_ids'])

# but list type은 파이토치 모델에 들어갈 수 없음

list

In [20]:
token = tokenizer(
    raw_text,
    return_tensors="pt" # 토큰화 결과 타입 지정 default : list, pt:pytorch, tf:tensorflow
)



In [24]:
type(token['input_ids'])

torch.Tensor

In [21]:
##############
# token 결과 -> model

# keyword 가변인자에 맞춰서 name=value 형태로 입력. input_ids=[.....]
# model에 맞춤 타입으로 입력(pytorch 모델 : torch.tensor)
context_vector = model(**token) # ert Model - > feature extract

In [ ]:
context_vector.last_hidden_state.shape
# 모든 입력 토큰들의 hidden state 들.
# context_vector 고정된 사이즈의 벡터

# [batch: 1, seq_len: 7, hidden_size(embeddingk vector): 768]
#                                    ㄴ 문맥 정보가 포함된 임베딩 백터 값. (BERT를 통과해서 나온 값임)

torch.Size([1, 7, 768])

In [ ]:
# 입력 문장(문서)에 대한 context vector - 문장(문서)의 특성값
## Bert 는 last_hidden_state에서 첫 번째 토큰([CLS]:문장의 시작을 알림)의 값을 가공(특성추출)해서 context vector로 사용
context_vector.pooler_output.shape

torch.Size([1, 768])

## kcbert
- BERT 모델을 한글 텍스트로 학습 시킨 Pretrained model.
    - BERT는 Transformer의 Encoder 부분을 이용해 구현된 언어모델
    - https://arxiv.org/abs/1810.04805 
- https://huggingface.co/beomi/kcbert-base

### 토크나이저, 모델 로드

In [27]:
from transformers import AutoModel, AutoTokenizer
model_id = "beomi/kcbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)

type(tokenizer), type(model)

(transformers.models.bert.tokenization_bert_fast.BertTokenizerFast,
 transformers.models.bert.modeling_bert.BertModel)

In [28]:
# 단일 문서 입력 : padding_pad 추가, truncation - 잘라내기 (토큰수 맞추는 작업.) 단일 문장에서는 신경 쓸 필요가 없다.
tokens = tokenizer(
    "나는 어제 친구와 밥을 먹었다.",
    return_tensors="pt"
)

tokens

{'input_ids': tensor([[    2,  8616,  9909,  9025,  4196, 23905, 21452,  4020,    17,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [31]:
tokenizer.encode("나는 어제 친구와 밥을 먹었다.") # imput_id만 출력
tokenizer.decode([2, 8616, 9909, 9025, 4196, 23905, 21452, 4020, 17, 3]) # token들 -> 문장(스페셜 토큰 포함)[CLS],[SEP]
tokenizer.decode(
    [2, 8616, 9909, 9025, 4196, 23905, 21452, 4020, 17, 3],
    skip_special_tokens=True
    )

'나는 어제 친구와 밥을 먹었다.'

In [35]:
# 개별 토큰 관련 조회
tokenizer.convert_ids_to_tokens(8616) #토큰 ID -> 토큰 문자열
tokenizer.convert_ids_to_tokens([9025, 4196, 23905]) #->['친구', '##와', '밥을']
tokenizer.convert_tokens_to_ids("어제")
tokenizer.convert_tokens_to_ids(["친구", "와", "밥을"])

[9025, 2320, 23905]

In [65]:
#토크나이저 전체 어휘 수
print(tokenizer.vocab_size)
tokenizer.get_vocab() # 어휘 사전 조회

30000


{'##구속': 13140,
 '##함을': 11091,
 '짯': 2721,
 '기자들': 13610,
 '전과': 14068,
 '것인지': 16216,
 '##쪽은': 16915,
 '구질': 23367,
 '##돕': 5229,
 '김종': 12952,
 '당첨': 22961,
 '##학': 4087,
 '풜': 3325,
 '어디까지': 13655,
 '문자': 15462,
 '깜': 436,
 '아내': 14074,
 '##종석': 12701,
 '🐛': 3687,
 '공론': 28949,
 '내로남불의': 17312,
 '에이': 10798,
 '매출': 13640,
 '##걸이': 13131,
 '뒷돈': 20398,
 '선장': 25953,
 '수용': 14358,
 '##시길': 9065,
 '🤘': 3938,
 '똑똑': 10055,
 '##교체': 16124,
 '꼽': 500,
 '##많다': 14484,
 '##십쇼': 19299,
 '꽌': 510,
 '구하라': 23846,
 '감염된': 23265,
 '쓰레기들이': 14348,
 '큿': 3093,
 '릴': 1283,
 '##격리': 11586,
 '욹': 2365,
 '눈앞에': 25138,
 '웃기다': 22659,
 '하러': 19669,
 '옮기': 24395,
 '##자인': 11026,
 '한강': 14102,
 '최성해': 23589,
 '갔으면': 22984,
 '에잇': 29212,
 '어이상실': 28825,
 '##따로': 21145,
 '뚤': 1076,
 '옹호하고': 21780,
 '캡': 3001,
 '##겠지요': 12380,
 '##닼': 5295,
 '아이들은': 19074,
 '고생이': 18860,
 '##🔉': 7738,
 '갔다가': 21784,
 '##쏘': 5052,
 '교수는': 25437,
 '##있냐': 9782,
 '지소미아': 11334,
 '##텃': 5679,
 '##장수': 27599,
 '가는지': 26571,
 '법대

In [ ]:
############################################
# 여러 문서 입력
############################################

In [36]:
sentences = [
    "안녕",
    "Hugging Face는 인공지능(AI)과 자연어 처리(NLP) 분야에서 혁신적인 도구와 모델을 제공하는 AI 스타트업이다.",
    "2016년에 설립된 이 회사는 주로 오픈소스 라이브러리와 사전 학습된 NLP 모델을 제공을 제공한다."
]

In [38]:
sent_tokens = tokenizer(
    sentences
    # return_tensors = "pt"
)

In [41]:
print(len(sent_tokens['input_ids']), sent_tokens['input_ids'])
print(len(sent_tokens['attention_mask']))
print(len(sent_tokens['token_type_ids']))

3 [[2, 19017, 3], [2, 41, 4223, 4403, 4403, 18940, 39, 4243, 4773, 4226, 4008, 14583, 25061, 11, 22502, 12, 321, 10459, 4071, 9810, 11, 47, 4450, 4579, 12, 16029, 7971, 13064, 8097, 867, 4228, 4196, 16505, 4027, 13248, 7966, 22502, 12296, 4104, 22192, 4020, 17, 3], [2, 26182, 4113, 20684, 4130, 2451, 22088, 20002, 15999, 4266, 4103, 17564, 4408, 4053, 4038, 4196, 11202, 20323, 4130, 47, 4450, 4579, 16505, 4027, 13248, 4027, 13248, 8008, 17, 3]]
3
3


In [42]:
# 토큰수
for ids in sent_tokens['input_ids']:
    print(len(ids))

3
43
30


In [ ]:
# max_length - 최대 길이(토큰수)를 지정.

# truncation - 길이를 밎추기 위해서 토큰의 일부를 잘라내는 것.
##           - True("longest") : 배치 중 가장 긴 문서에 맞춘다. max_length 설정이 된 경우에는 그에 맞춘다.
##           - False("do_not_truncate"): 기본값. Truncation을 안함

# padding - 길이를 맞추기 위해서 [PAD] 토큰으로 채우는 것.
##        -True("longest"): 배치 중에 가장 긴 문서에 맞춘다.
##        -"max_length": max_length 설정에 맞춘다. (max_length가 없으면 model의 디폴트 max_length에 맞춘다.)
##        -False("do_not_pad"): 기본값. Padding 처리하지 않는다.

# 순서 : truncation -> padding

In [63]:
sent_tokens2 = tokenizer(
    sentences,
    max_length = 20,
    padding= "max_length",
    truncation=True
)

In [64]:
for sent2 in sent_tokens2['input_ids']:
    print(len(sent2)) # sent)

20
20
20


In [55]:
sent_tokens2['attention_mask'][0]

[1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

### BERT 모델을 이용해 context vector 추출
#### Model 추론결과
- **last_hidden_state**
    - 모든 token들에 대한 feature
    - 출력이 **many**인 작업에 사용한다.
- **pooler_output**
    - 입력 문장, 텍스트에 대한 context vector 이다.
    - 이 값은 **문장을 입력받아 처리하는 task**(ex: 문서분류-감정분석,문장카테고리분류, 문장유사도 분석)의 입력으로 사용한다.

In [69]:
tokens = tokenizer(
    sentences,
    max_length = 20,
    padding= True,
    truncation=True,
    return_tensors="pt"
)

In [70]:
result = model(**tokens)

In [71]:
result.last_hidden_state.shape

torch.Size([3, 20, 768])

In [72]:
result.pooler_output.shape

torch.Size([3, 768])